In [4]:
import pyhdb
import time
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import csv
import os, re

In [6]:
def GetHADConnection(): # developement DataBase
    conn = pyhdb.connect(host="bishdbd01",
                         port=31015,
                         user="ZMLF_FICO_COPATMA",
                         password="Hanapod10")
    return conn

def GetHAPConnection(): # production DataBase
    conn = pyhdb.connect(host="bishdbp01",
                          port=34015,
                          user="ZMLF_FICO_COPATMA",
                          password="Hanapod10")
    return conn

def GetQuery1Master():
    sql_query = """SELECT SITE, UPC, CLIENT_ID FROM "MIQ"."MIQ_POS_ONLY" 
                WHERE (FISCAL_YEAR_WEEK >= 201910)
                AND (FISCAL_YEAR_WEEK <= 201915) AND (CLIENT_ID = 'LCL')"""
    conn1  = GetHAPConnection()
    reg_crsr = conn1.cursor()
    reg_crsr.execute(sql_query)
    prod_map_data = reg_crsr.fetchall()
    dquery   = pd.DataFrame(prod_map_data)
    cols  = [column[0] for column in reg_crsr.description]
    for i in range(0,len(cols)): dquery = dquery.rename(columns={i:cols[i]})
    
    return dquery

def GetStoreMaster():
    sql_query = """
                    SELECT
                    ZCSTORID AS STORE_ID,
                    ZCCURBAN AS BANNER,
                    ZCMLFREGN AS REGION,
                    "0LONGITUDE" AS LONGITUDE,
                    "0LATITUDE" AS LATITUDE,
                    ZCPOSADDR AS ADDRESS,
                    ZCPOSCITY AS CITY,
                    ZCPOSPROV AS PROVINCE,
                    ZCPOSTLCD AS POSTAL_CODE
                    FROM "MIQ"."MIQ_ZOSCCU01"
                    WHERE ZCCURSTOR=ZCSTORNUM """

    conn1  = GetHAPConnection()
    reg_crsr = conn1.cursor()
    reg_crsr.execute(sql_query)
    prod_map_data = reg_crsr.fetchall()
    dstore   = pd.DataFrame(prod_map_data)
    cols  = [column[0] for column in reg_crsr.description]
    for i in range(0,len(cols)): dstore = dstore.rename(columns={i:cols[i]})
    
    return dstore

def GetProdMaster():
    sql_prod ="SELECT MATERIAL AS SAP_MATERIAL, PRODH6 AS SEGMENT FROM MIQ.MIQ_ZOFNSGDD"
    conn1  = GetHAPConnection()
    reg_crsr = conn1.cursor()
    reg_crsr.execute(sql_prod)
    prod_map_data = reg_crsr.fetchall()
    dprod   = pd.DataFrame(prod_map_data)
    cols  = [column[0] for column in reg_crsr.description]
    for i in range(0,len(cols)): dprod = dprod.rename(columns={i:cols[i]})
    dprod['SAP_MATERIAL']= dprod['SAP_MATERIAL'].astype('int64')
    return dprod

def GetCustMaster():

    sql_clt_bnr =  """SELECT ZCSTORID AS STORE_ID,
                ZCCURBAN AS BANNER,
                ZCMLFREGN AS REGION
                FROM "MIQ"."MIQ_ZOSCCU01"
                WHERE ZCCURSTOR=ZCSTORNUM
                    """

    conn1  = GetHAPConnection()
    crsr  = conn1.cursor()
    reg_crsr = conn1.cursor()
    reg_crsr.execute(sql_clt_bnr)
    regmap_data = reg_crsr.fetchall()
    dcust  = pd.DataFrame(regmap_data)
    cols  = [column[0] for column in reg_crsr.description]
    for i in range(0,len(cols)):dcust =dcust.rename(columns={i:cols[i]})
    dcust['CLIENT_ID']=dcust['STORE_ID'].str[:3]
    dcust['STORE_NUMBER']=dcust['STORE_ID'].str[3:]

    return dcust

# AND (CLIENT_ID ='"""+str(client)+"""') 

def GetPOSData(client, start_week,end_week):
    dprod = GetProdMaster()
    dcust = GetCustMaster()
    sql_pos = """SELECT 
            SAP_MATERIAL,
            CLIENT_ID,
            STORE_NUM AS STORE_NUMBER,
            MLF_WEEK,
            POS_QTY,
            POS_QTY_BASE,
            POS_QTY_LIFT
            FROM MIQ.MIQ_POS_TS 
            WHERE ((MLF_WEEK >= """+str(start_week)+""")
            AND (MLF_WEEK <= """+str(end_week)+""")
            AND (CLIENT_ID ='"""+str(client)+"""')
            AND (POS_QTY IS NOT NULL))"""
    conn1  = GetHAPConnection()
    crsr  = conn1.cursor()
    reg_crsr = conn1.cursor()
    reg_crsr.execute(sql_pos)
    regmap_data = reg_crsr.fetchall()
    dpos  = pd.DataFrame(regmap_data)
    cols  = [column[0] for column in reg_crsr.description]
    for i in range(0,len(cols)):dpos =dpos.rename(columns={i:cols[i]})
    dpos = dpos.merge(dcust,on=['CLIENT_ID','STORE_NUMBER'],how='left')
    dpos = dpos[dpos['SAP_MATERIAL'].notnull()]
    dpos['SAP_MATERIAL']= dpos['SAP_MATERIAL'].astype('int64')
    dPOS = dpos.merge(dprod,on=['SAP_MATERIAL'],how='left')
    dPOS = dPOS[dPOS['POS_QTY_LIFT'].notnull()]
    #dPOS = dPOS.groupby(['BANNER', 'REGION', 'SEGMENT', 'MLF_WEEK'])['POS_QTY', 'POS_QTY_BASE', 'POS_QTY_LIFT'].sum().reset_index()
    dPOS['SEGMENT'] = dPOS['SEGMENT'].astype(str)
    return dPOS


In [11]:
#dquery = GetQuery1Master()
#dquery

In [8]:
dstore = GetStoreMaster()
len(dstore[dstore["BANNER"]== "COSTCO"])

112